In [ ]:
!pip install requests beautifulsoup4 --quiet

import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime

GREEN = '\033[92m'
RED = '\033[91m'
YELLOW = '\033[93m'
RESET = '\033[0m'

def banner():
    print(f"""{GREEN}
    ╔──────────────────────────────────────────────╗
    │      Website Vulnerability Scanner           │
    │        SQLi - XSS - HTTPS Check              │
    ╚──────────────────────────────────────────────╝{RESET}
    """)

def check_https(url):
    try:
        if not url.startswith(("http://", "https://")):
            url = "http://" + url
        response = requests.get(url, timeout=10, verify=True)
        if response.url.startswith("https://"):
            return f"{GREEN}HTTPS مستخدم وشهادة SSL سليمة{RESET}"
        else:
            return f"{YELLOW}الموقع يستخدم HTTP فقط - عرضة لـ MITM{RESET}"
    except requests.exceptions.SSLError:
        return f"{RED}شهادة SSL منتهية أو غير موثوقة{RESET}"
    except:
        return f"{RED}مشكلة في الاتصال أو SSL{RESET}"

def check_sql_injection(url):
    payloads = ["' OR '1'='1", "' OR '1'='1' --", "admin' --", "' UNION SELECT NULL--"]
    for payload in payloads:
        try:
            test_url = f"{url}?id={payload}" if "id=" in url or "?" in url else f"{url}?q={payload}"
            r = requests.get(test_url, timeout=8)
            if any(x in r.text.lower() for x in ["sql", "syntax", "mysql", "sqlite", "warning"]):
                return f"{RED}ثغرة SQL Injection مكتشفة! (Payload: {payload}){RESET}"
        except:
            pass
    return f"{GREEN}لا توجد ثغرة SQL Injection واضحة{RESET}"

def check_xss(url):
    payload = "<script>alert('XSS_By_YourName')</script>"
    try:
        test_url = f"{url}?q={payload}" if "?" in url else f"{url}?search={payload}"
        r = requests.get(test_url, timeout=8)
        if payload in r.text:
            return f"{RED}ثغرة XSS مكتشفة! الكود ظهر في الصفحة{RESET}"
    except:
        pass
    return f"{GREEN}لا توجد ثغرة XSS واضحة{RESET}"

# اختبار سريع (غيّر الرابط لأي موقع تجريبي)
banner()
url = "http://demo.testfire.net/"   # موقع تجريبي شهير وآمن 100%

print(f"جاري فحص: {url}\n")
print(check_https(url))
print(check_sql_injection(url))
print(check_xss(url))


    ╔──────────────────────────────────────────────╗
    │      Website Vulnerability Scanner           │
    │        SQLi - XSS - HTTPS Check              │
    ╚──────────────────────────────────────────────╝
    
جاري فحص: http://demo.testfire.net/

الموقع يستخدم HTTP فقط - عرضة لـ MITM
لا توجد ثغرة SQL Injection واضحة
لا توجد ثغرة XSS واضحة


In [ ]:
# تثبيت المكتبات (مرة واحدة بس)
!pip install requests beautifulsoup4 --quiet

import requests
from bs4 import BeautifulSoup
from datetime import datetime

# ألوان للجمال
GREEN = '\033[92m'
RED = '\033[91m'
YELLOW = '\033[93m'
RESET = '\033[0m'

print(f"""{GREEN}
╔──────────────────────────────────────────────────────╗
│      Website Vulnerability Scanner - نسخة تفاعلية      │
│           SQLi • XSS • HTTPS Check                   │
╚──────────────────────────────────────────────────────╝{RESET}\n""")

# هنا المستخدم يكتب الرابط بنفسه
url = input("أدخل رابط الموقع اللي عايز تفحصه (مثال: http://testphp.vulnweb.com): ").strip()

if not url:
    print(f"{RED}يجب إدخال رابط!{RESET}")
    exit()

if not url.startswith(("http://", "https://")):
    url = "http://" + url

print(f"\nجاري فحص الموقع: {url}\n{'─'*60}")

def check_https(url):
    try:
        response = requests.get(url, timeout=12, verify=True)
        if response.url.startswith("https://"):
            print(f"{GREEN}HTTPS مستخدم وشهادة SSL سليمة{RESET}")
        else:
            print(f"{YELLOW}تحذير: الموقع يعمل بـ HTTP فقط → عرضة لهجمات Man-in-the-Middle{RESET}")
    except requests.exceptions.SSLError:
        print(f"{RED}مشكلة في شهادة SSL (منتهية أو غير موثوقة){RESET}")
    except:
        print(f"{RED}لا يمكن الوصول للموقع{RESET}")

def check_sql_injection(url):
    payloads = ["' OR '1'='1", "' OR '1'='1' --", "admin' --", "' UNION SELECT NULL--"]
    for payload in payloads:
        try:
            test_url = f"{url}?id={payload}" if "?" in url else f"{url}?q={payload}"
            r = requests.get(test_url, timeout=8)
            if any(keyword in r.text.lower() for keyword in ["sql", "syntax", "mysql", "error", "warning"]):
                print(f"{RED}ثغرة SQL Injection مكتشفة باستخدام: {payload}{RESET}")
                return
        except:
            pass
    print(f"{GREEN}لا توجد ثغرة SQL Injection واضحة{RESET}")

def check_xss(url):
    payload = "<script>alert('XSS_By_YourName')</script>"
    try:
        test_url = f"{url}?q={payload}" if "?" in url else f"{url}?search={payload}"
        r = requests.get(test_url, timeout=8)
        if payload in r.text or "XSS_By_YourName" in r.text:
            print(f"{RED}ثغرة XSS مكتشفة → الكود الضار ظهر في الصفحة!{RESET}")
            return
    except:
        pass
    print(f"{GREEN}لا توجد ثغرة XSS واضحة{RESET}")

# تنفيذ الفحوصات
check_https(url)
check_sql_injection(url)
check_xss(url)

# حفظ تقرير تلقائي
report_name = "scan_report.txt"
with open(report_name, "w", encoding="utf-8") as f:
    f.write(f"تقرير فحص أمني - {datetime.now().strftime('%Y-%m-%d %H:%M')}\n")
    f.write(f"الهدف: {url}\n{'═'*50}\n")
    f.write("تم الفحص بنجاح عبر الأداة المطورة\n")

print(f"\n{GREEN}تم حفظ التقرير في ملف: {report_name}{RESET}")
print(f"{GREEN}المشروع جاهز للتسليم!{RESET}")


╔──────────────────────────────────────────────────────╗
│      Website Vulnerability Scanner - نسخة تفاعلية      │
│           SQLi • XSS • HTTPS Check                   │
╚──────────────────────────────────────────────────────╝

أدخل رابط الموقع اللي عايز تفحصه (مثال: http://testphp.vulnweb.com): http://testphp.vulnweb.com

جاري فحص الموقع: http://testphp.vulnweb.com
────────────────────────────────────────────────────────────
تحذير: الموقع يعمل بـ HTTP فقط → عرضة لهجمات Man-in-the-Middle
ثغرة SQL Injection مكتشفة باستخدام: ' OR '1'='1
لا توجد ثغرة XSS واضحة

تم حفظ التقرير في ملف: scan_report.txt
المشروع جاهز للتسليم!


In [ ]:
# تثبيت كل المكتبات المطلوبة مرة واحدة
!pip install requests beautifulsoup4 fpdf2 ipywidgets --quiet

import requests
from bs4 import BeautifulSoup
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from fpdf import FPDF
from datetime import datetime
import io

# ألوان HTML للواجهة
STYLE = """
<style>
    .title {font-size: 28px; font-weight: bold; color: #d32f2f; text-align: center; margin: 20px;}
    .subtitle {font-size: 18px; color: #1976d2; text-align: center;}
    .result-good {color: green; font-weight: bold;}
    .result-bad {color: red; font-weight: bold;}
    .result-warn {color: orange; font-weight: bold;}
</style>
"""

display(HTML(STYLE + """
<div class="title">Website Vulnerability Scanner</div>
<div class="subtitle">فاحص ثغرات أمنية تفاعلي - SQLi • XSS • HTTPS</div>
"""))

# حقل إدخال الرابط
url_input = widgets.Text(
    value='http://testphp.vulnweb.com',
    placeholder='أدخل رابط الموقع هنا',
    description='الموقع:',
    layout=widgets.Layout(width='80%'),
    style={'description_width': '80px'}
)

# زر الفحص
scan_button = widgets.Button(
    description='ابدأ الفحص',
    button_style='danger',
    layout=widgets.Layout(width='200px', height='50px')
)

# مكان إظهار النتايج
output = widgets.Output()

def run_scan(b):
    with output:
        clear_output()
        url = url_input.value.strip()
        if not url:
            print("خطأ: أدخل رابط صحيح!")
            return

        if not url.startswith(("http://", "https://")):
            url = "http://" + url

        print(f"جاري فحص: {url}\n{'═'*60}")
        results = []
        results.append(f"تقرير فحص أمني\nالتاريخ: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
        results.append(f"الهدف: {url}\n{'─'*60}")

        # 1. فحص HTTPS
        try:
            r = requests.get(url, timeout=12, verify=True)
            if r.url.startswith("https://"):
                print("HTTPS مستخدم وشهادة SSL سليمة")
                results.append("HTTPS: آمن")
            else:
                print("تحذير: الموقع يعمل بـ HTTP فقط → عرضة للاختراق")
                results.append("HTTPS: غير آمن (HTTP)")
        except requests.exceptions.SSLError:
            print("مشكلة في شهادة SSL")
            results.append("HTTPS: شهادة منتهية أو غير موثوقة")
        except:
            print("لا يمكن الوصول للموقع")
            results.append("HTTPS: لا يمكن الوصول")

        # 2. فحص SQL Injection
        payloads = ["' OR '1'='1", "' OR '1'='1' --", "admin' --"]
        sqli_found = False
        for p in payloads:
            try:
                test_url = f"{url}?id={p}" if "?" in url else f"{url}?q={p}"
                resp = requests.get(test_url, timeout=8)
                if any(k in resp.text.lower() for k in ["sql", "syntax", "mysql", "error"]):
                    print(f"ثغرة SQL Injection مكتشفة! (Payload: {p})")
                    results.append(f"SQL Injection: موجودة → {p}")
                    sqli_found = True
                    break
            except:
                pass
        if not sqli_found:
            print("لا توجد ثغرة SQL Injection واضحة")
            results.append("SQL Injection: غير موجودة")

        # 3. فحص XSS
        xss_payload = "<script>alert('XSS_By_YourName')</script>"
        try:
            test_url = f"{url}?q={xss_payload}" if "?" in url else f"{url}?search={xss_payload}"
            resp = requests.get(test_url, timeout=8)
            if xss_payload in resp.text:
                print("ثغرة XSS مكتشفة → الكود نفّذ!")
                results.append("XSS: موجودة")
            else:
                print("لا توجد ثغرة XSS واضحة")
                results.append("XSS: غير موجودة")
        except:
            print("لا يمكن اختبار XSS")
            results.append("XSS: غير قابل للفحص")

        print(f"\n{'═'*60}\nالفحص انتهى!")

        # زر تحميل التقرير PDF
        pdf = FPDF()
        pdf.add_page()
        pdf.add_font("DejaVu", "", "DejaVuSans.ttf", uni=True)  # خط عربي
        pdf.set_font("DejaVu", size=12)
        pdf.cell(200, 10, txt="تقرير فحص أمني", ln=1, align='C')
        pdf.cell(200, 10, txt=f"التاريخ: {datetime.now().strftime('%Y-%m-%d %H:%M')}", ln=1)
        pdf.cell(200, 10, txt=f"الموقع: {url}", ln=2)
        for line in results[2:]:
            pdf.cell(200, 10, txt=line, ln=1)

        pdf_output = io.BytesIO()
        pdf.output(pdf_output)
        pdf_data = pdf_output.getvalue()

        download_btn = widgets.Button(description="تحميل التقرير PDF")
        def download_pdf(b):
            from google.colab import files
            with open("تقرير_الفحص.pdf", "wb") as f:
                f.write(pdf_data)
            files.download("تقرير_الفحص.pdf")

        download_btn.on_click(download_pdf)
        display(download_btn)

# ربط الزر بالدالة
scan_button.on_click(run_scan)

# عرض الواجهة
display(url_input)
display(scan_button)
display(output)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.8 MB/s eta 0:00:00


Text(value='http://testphp.vulnweb.com', description='الموقع:', layout=Layout(width='80%'), placeholder='أدخل …

Button(button_style='danger', description='ابدأ الفحص', layout=Layout(height='50px', width='200px'), style=But…

Output()

In [ ]:

import requests
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from datetime import datetime
import io

# ==============================================================================
# 1. تصميم واجهة المستخدم (GUI)
# ==============================================================================

# تعريف أنماط CSS لتحسين مظهر الواجهة وعرض النتائج
STYLE = """
<style>
    /* الخطوط والألوان العامة */
    @import url('https://fonts.googleapis.com/css2?family=Cairo:wght@400;700&display=swap');
    body, .widget-box {
        font-family: 'Cairo', sans-serif;
    }

    /* العنوان الرئيسي */
    .title {
        font-size: 32px;
        font-weight: 700;
        color: #004d40; /* Teal Dark */
        text-align: center;
        margin: 20px 0;
        border-bottom: 3px solid #004d40;
        padding-bottom: 10px;
    }

    /* العنوان الفرعي */
    .subtitle {
        font-size: 18px;
        color: #26a69a; /* Teal Light */
        text-align: center;
        margin-bottom: 20px;
    }

    /* أنماط النتائج */
    .result-good {color: #388e3c; font-weight: bold;} /* Green */
    .result-bad {color: #d32f2f; font-weight: bold;} /* Red */
    .result-warn {color: #fbc02d; font-weight: bold;} /* Amber */

    /* بطاقة النتيجة */
    .scan-card {
        padding: 15px;
        margin: 10px 0;
        border-radius: 8px;
        box-shadow: 0 4px 8px rgba(0,0,0,0.1);
        border-right: 5px solid;
    }
    .card-safe { border-right-color: #388e3c; background-color: #e8f5e9; }
    .card-danger { border-right-color: #d32f2f; background-color: #ffebee; }
    .card-warning { border-right-color: #fbc02d; background-color: #fffde7; }
    .card-info { border-right-color: #1976d2; background-color: #e3f2fd; }

    .scan-card h4 {
        margin-top: 0;
        border-bottom: 1px dashed #ccc;
        padding-bottom: 5px;
        font-weight: 700;
        color: #455a64;
    }
</style>
"""

# عرض العناوين باستخدام HTML المخصص
display(HTML(STYLE))
display(HTML('<div class="title">ماسح الثغرات الأمنية للمواقع</div>'))
display(HTML('<div class="subtitle">فحص أولي للثغرات الشائعة: SQLi • XSS • HTTPS</div>'))


# حقل إدخال الرابط
url_input = widgets.Text(
    value='http://testphp.vulnweb.com',
    placeholder='أدخل الرابط الكامل للموقع هنا (مثال: https://example.com)',
    description='رابط الموقع:',
    layout=widgets.Layout(width='90%'),
    style={'description_width': '120px'}
)

# زر الفحص
scan_button = widgets.Button(
    description='بدء الفحص الأمني',
    button_style='info', # تغيير النمط ليتناسب مع التصميم
    icon='search',
    layout=widgets.Layout(width='90%', height='50px')
)

# مكان إظهار النتائج
output = widgets.Output()

# تجميع حقل الإدخال والزر في صندوق رأسي
input_box = widgets.VBox([url_input, scan_button], layout=widgets.Layout(align_items='center', margin='10px'))
display(input_box)
display(output)


# ==============================================================================
# 2. منطق الفحص
# ==============================================================================

def create_report_card(title, result, status_class, description):
    """إنشاء بطاقة HTML لعرض نتيجة فحص معين."""
    return f"""
    <div class="scan-card {status_class}">
        <h4>{title} <span style="float: left;"> الحالة: <span class="{status_class.replace('card-', 'result-')}">{result}</span></span></h4>
        <p>{description}</p>
    </div>
    """

def run_scan(b):
    """تشغيل عملية الفحص عند النقر على الزر."""
    with output:
        clear_output()
        url = url_input.value.strip()

        if not url:
            display(HTML('<div class="result-bad">خطأ: الرجاء إدخال رابط صحيح للموقع!</div>'))
            return

        # تصحيح البروتوكول في حال نسيانه
        if not url.startswith(("http://", "https://")):
            url = "http://" + url

        display(HTML(f"<h3><span class='result-warn'>جاري فحص:</span> {url}</h3>"))

        # قائمة لتجميع بطاقات النتائج لعرضها مرة واحدة
        html_results = []

        # ======================================================================
        # 1. فحص الاتصال الأولي و HTTPS
        # ======================================================================

        try:
            # محاولة الوصول عبر HTTPS أولاً
            r = requests.get(url, timeout=15, verify=True, allow_redirects=True)

            # التحقق من أن الموقع يعمل بـ HTTPS بعد إعادة التوجيه
            if r.url.startswith("https://"):
                html_results.append(create_report_card(
                    "فحص أمان الاتصال (HTTPS)",
                    "آمن",
                    "card-safe",
                    "الموقع يعمل ببروتوكول HTTPS المشفر، مما يضمن سرية البيانات المنقولة."
                ))
            else:
                html_results.append(create_report_card(
                    "فحص أمان الاتصال (HTTPS)",
                    "تحذير",
                    "card-warning",
                    "الموقع يستخدم بروتوكول HTTP غير المشفر. قد تكون البيانات عرضة للاعتراض."
                ))

        except requests.exceptions.SSLError:
            # مشكلة في شهادة SSL
            html_results.append(create_report_card(
                "فحص أمان الاتصال (HTTPS)",
                "خطأ في الشهادة",
                "card-danger",
                "تم اكتشاف مشكلة في شهادة SSL (مثل انتهاء الصلاحية أو عدم الموثوقية). يجب إصلاحها فوراً."
            ))
        except requests.exceptions.RequestException as e:
            # أي خطأ آخر في الاتصال (مثل عدم الوصول للموقع)
            html_results.append(create_report_card(
                "فحص الاتصال",
                "فشل",
                "card-danger",
                f"فشل الاتصال بالموقع الهدف. (الخطأ: {str(e)})"
            ))
            # إنهاء الفحص إذا لم يتمكن من الوصول للموقع
            display(HTML("".join(html_results)))
            display(HTML(f"<h4><span class='result-bad'>الفحص توقف:</span> تعذر الوصول للهدف.</h4>"))
            return

        # ======================================================================
        # 2. فحص حقن SQL (SQLi)
        # ======================================================================

        sqli_payloads = ["' OR '1'='1", "1' OR '1'='1' --", "admin' --"]
        sqli_found = False

        # محاولة فحص المسارات الشائعة للاستعلامات
        test_url_base = url.split('?')[0] # إزالة أي معلمات URL موجودة
        test_paths = ['?id=', '?q=', '/product?id=', '/item?q=']

        for p in sqli_payloads:
            for path in test_paths:
                test_url = test_url_base + path + p
                try:
                    resp = requests.get(test_url, timeout=8)
                    # البحث عن كلمات مفتاحية تشير إلى أخطاء في قاعدة البيانات
                    if any(k in resp.text.lower() for k in ["sql", "syntax", "mysql", "error", "warning"]):
                        html_results.append(create_report_card(
                            "فحص حقن SQL (SQLi)",
                            "مكتشفة (خطير)",
                            "card-danger",
                            f"تم اكتشاف ثغرة حقن SQL واضحة! (Payload: <code>{p}</code>). قد يتمكن المهاجم من الوصول لقاعدة البيانات."
                        ))
                        sqli_found = True
                        break # الخروج من حلقة الـ path بعد الإيجاد
                except requests.exceptions.RequestException:
                    continue # تجاهل أخطاء الاتصال للمسارات الفرعية
            if sqli_found:
                break # الخروج من حلقة الـ payload بعد الإيجاد

        if not sqli_found:
            html_results.append(create_report_card(
                "فحص حقن SQL (SQLi)",
                "غير موجودة",
                "card-safe",
                "لم يتم العثور على دلائل واضحة لثغرة SQLi بالحمولة الأولية المستخدمة."
            ))

        # ======================================================================
        # 3. فحص البرمجة عبر المواقع (XSS)
        # ======================================================================

        # حمولة XSS أساسية لاختبار انعكاس النص
        xss_payload = "<script>console.log('XSS_TEST_REFLECTION')</script>"
        xss_test_string = "XSS_TEST_REFLECTION"
        xss_found = False

        # محاولة إدخال الحمولة في مسار افتراضي
        test_url = test_url_base + f"?search={xss_payload}"
        try:
            resp = requests.get(test_url, timeout=8)
            # التحقق مما إذا تم عكس النص داخل محتوى الصفحة
            if xss_test_string in resp.text:
                html_results.append(create_report_card(
                    "فحص البرمجة عبر المواقع (XSS)",
                    "مكتشفة (تحذير)",
                    "card-warning",
                    f"تم اكتشاف انعكاس محتمل لحمولة XSS (<code>{xss_payload}</code>) في الصفحة. قد يستغلها المهاجم."
                ))
                xss_found = True

        except requests.exceptions.RequestException:
            pass # تجاهل الخطأ والمتابعة

        if not xss_found:
            html_results.append(create_report_card(
                "فحص البرمجة عبر المواقع (XSS)",
                "غير موجودة",
                "card-safe",
                "لم يتم العثور على انعكاس مباشر لحمولة XSS النصية."
            ))

        # ======================================================================
        # 4. عرض ملخص النتائج
        # ======================================================================

        display(HTML("".join(html_results)))
        display(HTML(f"<h4><span class='result-good'>الفحص انتهى بنجاح!</span> يرجى مراجعة البطاقات أعلاه.</h4>"))


# ==============================================================================
# 3. ربط الدالة بالزر
# ==============================================================================
scan_button.on_click(run_scan)

Output()

In [ ]:

import requests
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from datetime import datetime
import io

# ==============================================================================
# 1. تصميم واجهة المستخدم (GUI)
# ==============================================================================

# تعريف أنماط CSS لتحسين مظهر الواجهة وعرض النتائج
STYLE = """
<style>
    /* الخطوط والألوان العامة */
    @import url('https://fonts.googleapis.com/css2?family=Cairo:wght@400;700&display=swap');
    body, .widget-box {
        font-family: 'Cairo', sans-serif;
    }

    /* العنوان الرئيسي - لون أزرق غامق جديد */
    .title {
        font-size: 32px;
        font-weight: 700;
        color: #1a237e; /* Indigo Dark */
        text-align: center;
        margin: 20px 0;
        border-bottom: 3px solid #1a237e;
        padding-bottom: 10px;
    }

    /* العنوان الفرعي - لون أزرق فاتح جديد */
    .subtitle {
        font-size: 18px;
        color: #5c6bc0; /* Indigo Light */
        text-align: center;
        margin-bottom: 20px;
    }

    /* أنماط النتائج */
    /* أخضر ساطع للآمن */
    .result-good {color: #2e7d32; font-weight: bold;}
    /* أحمر داكن للخطير */
    .result-bad {color: #c62828; font-weight: bold;}
    /* برتقالي للتحذير */
    .result-warn {color: #ff8f00; font-weight: bold;}

    /* بطاقة النتيجة */
    .scan-card {
        padding: 15px;
        margin: 10px 0;
        border-radius: 8px;
        box-shadow: 0 4px 10px rgba(0,0,0,0.15); /* ظل أوضح */
        border-right: 7px solid; /* حد أسمك */
        transition: transform 0.2s;
    }
    .scan-card:hover {
        transform: translateY(-2px);
    }

    /* خلفيات وألوان الحدود الجديدة */
    .card-safe { border-right-color: #43a047; background-color: #f1f8e9; } /* أخضر فاتح جداً */
    .card-danger { border-right-color: #e53935; background-color: #ffebee; } /* أحمر فاتح جداً */
    .card-warning { border-right-color: #fb8c00; background-color: #fff8e1; } /* أصفر فاتح جداً */
    .card-info { border-right-color: #1e88e5; background-color: #e3f2fd; } /* أزرق فاتح للمعلومات */

    .scan-card h4 {
        margin-top: 0;
        border-bottom: 1px dashed #bdbdbd;
        padding-bottom: 5px;
        font-weight: 700;
        color: #37474f; /* أزرق رمادي داكن */
    }
    /* تحسين زر الفحص */
    .widget-button {
        background-color: #3f51b5 !important; /* أزرق نيلي جذاب */
        color: white !important;
        font-weight: bold !important;
        border-radius: 25px !important;
        border: none !important;
        box-shadow: 0 4px 8px rgba(63, 81, 181, 0.4) !important;
    }
</style>
"""

# عرض العناوين باستخدام HTML المخصص
display(HTML(STYLE))
display(HTML('<div class="title">ماسح الثغرات الأمنية للمواقع</div>'))
display(HTML('<div class="subtitle">فحص أولي للثغرات الشائعة: SQLi • XSS • HTTPS</div>'))


# حقل إدخال الرابط
url_input = widgets.Text(
    value='http://testphp.vulnweb.com',
    placeholder='أدخل الرابط الكامل للموقع هنا (مثال: https://example.com)',
    description='رابط الموقع:',
    layout=widgets.Layout(width='90%'),
    style={'description_width': '120px'}
)

# زر الفحص
scan_button = widgets.Button(
    description='بدء الفحص الأمني',
    button_style='info',
    icon='search',
    layout=widgets.Layout(width='90%', height='50px')
)
# إضافة الكلاس المخصص للزر ليتأثر بالتصميم الجديد
scan_button.add_class('widget-button')


# مكان إظهار النتائج
output = widgets.Output()

# تجميع حقل الإدخال والزر في صندوق رأسي
input_box = widgets.VBox([url_input, scan_button], layout=widgets.Layout(align_items='center', margin='10px'))
display(input_box)
display(output)


# ==============================================================================
# 2. منطق الفحص
# ==============================================================================

def create_report_card(title, result, status_class, description):
    """إنشاء بطاقة HTML لعرض نتيجة فحص معين."""
    return f"""
    <div class="scan-card {status_class}">
        <h4>{title} <span style="float: left;"> الحالة: <span class="{status_class.replace('card-', 'result-')}">{result}</span></span></h4>
        <p>{description}</p>
    </div>
    """

def run_scan(b):
    """تشغيل عملية الفحص عند النقر على الزر."""
    with output:
        clear_output()
        url = url_input.value.strip()

        if not url:
            display(HTML('<div class="result-bad">خطأ: الرجاء إدخال رابط صحيح للموقع!</div>'))
            return

        # تصحيح البروتوكول في حال نسيانه
        if not url.startswith(("http://", "https://")):
            url = "http://" + url

        display(HTML(f"<h3><span class='result-warn'>جاري فحص:</span> {url}</h3>"))

        # قائمة لتجميع بطاقات النتائج لعرضها مرة واحدة
        html_results = []

        # ======================================================================
        # 1. فحص الاتصال الأولي و HTTPS
        # ======================================================================

        try:
            # محاولة الوصول عبر HTTPS أولاً
            r = requests.get(url, timeout=15, verify=True, allow_redirects=True)

            # التحقق من أن الموقع يعمل بـ HTTPS بعد إعادة التوجيه
            if r.url.startswith("https://"):
                html_results.append(create_report_card(
                    "فحص أمان الاتصال (HTTPS)",
                    "آمن",
                    "card-safe",
                    "الموقع يعمل ببروتوكول HTTPS المشفر، مما يضمن سرية البيانات المنقولة."
                ))
            else:
                html_results.append(create_report_card(
                    "فحص أمان الاتصال (HTTPS)",
                    "تحذير",
                    "card-warning",
                    "الموقع يستخدم بروتوكول HTTP غير المشفر. قد تكون البيانات عرضة للاعتراض."
                ))

        except requests.exceptions.SSLError:
            # مشكلة في شهادة SSL
            html_results.append(create_report_card(
                "فحص أمان الاتصال (HTTPS)",
                "خطأ في الشهادة",
                "card-danger",
                "تم اكتشاف مشكلة في شهادة SSL (مثل انتهاء الصلاحية أو عدم الموثوقية). يجب إصلاحها فوراً."
            ))
        except requests.exceptions.RequestException as e:
            # أي خطأ آخر في الاتصال (مثل عدم الوصول للموقع)
            html_results.append(create_report_card(
                "فحص الاتصال",
                "فشل",
                "card-danger",
                f"فشل الاتصال بالموقع الهدف. (الخطأ: {str(e)})"
            ))
            # إنهاء الفحص إذا لم يتمكن من الوصول للموقع
            display(HTML("".join(html_results)))
            display(HTML(f"<h4><span class='result-bad'>الفحص توقف:</span> تعذر الوصول للهدف.</h4>"))
            return

        # ======================================================================
        # 2. فحص حقن SQL (SQLi)
        # ======================================================================

        sqli_payloads = ["' OR '1'='1", "1' OR '1'='1' --", "admin' --"]
        sqli_found = False

        # محاولة فحص المسارات الشائعة للاستعلامات
        test_url_base = url.split('?')[0] # إزالة أي معلمات URL موجودة
        test_paths = ['?id=', '?q=', '/product?id=', '/item?q=']

        for p in sqli_payloads:
            for path in test_paths:
                test_url = test_url_base + path + p
                try:
                    resp = requests.get(test_url, timeout=8)
                    # البحث عن كلمات مفتاحية تشير إلى أخطاء في قاعدة البيانات
                    if any(k in resp.text.lower() for k in ["sql", "syntax", "mysql", "error", "warning"]):
                        html_results.append(create_report_card(
                            "فحص حقن SQL (SQLi)",
                            "مكتشفة (خطير)",
                            "card-danger",
                            f"تم اكتشاف ثغرة حقن SQL واضحة! (Payload: <code>{p}</code>). قد يتمكن المهاجم من الوصول لقاعدة البيانات."
                        ))
                        sqli_found = True
                        break # الخروج من حلقة الـ path بعد الإيجاد
                except requests.exceptions.RequestException:
                    continue # تجاهل أخطاء الاتصال للمسارات الفرعية
            if sqli_found:
                break # الخروج من حلقة الـ payload بعد الإيجاد

        if not sqli_found:
            html_results.append(create_report_card(
                "فحص حقن SQL (SQLi)",
                "غير موجودة",
                "card-safe",
                "لم يتم العثور على دلائل واضحة لثغرة SQLi بالحمولة الأولية المستخدمة."
            ))

        # ======================================================================
        # 3. فحص البرمجة عبر المواقع (XSS)
        # ======================================================================

        # حمولة XSS أساسية لاختبار انعكاس النص
        xss_payload = "<script>console.log('XSS_TEST_REFLECTION')</script>"
        xss_test_string = "XSS_TEST_REFLECTION"
        xss_found = False

        # محاولة إدخال الحمولة في مسار افتراضي
        test_url = test_url_base + f"?search={xss_payload}"
        try:
            resp = requests.get(test_url, timeout=8)
            # التحقق مما إذا تم عكس النص داخل محتوى الصفحة
            if xss_test_string in resp.text:
                html_results.append(create_report_card(
                    "فحص البرمجة عبر المواقع (XSS)",
                    "مكتشفة (تحذير)",
                    "card-warning",
                    f"تم اكتشاف انعكاس محتمل لحمولة XSS (<code>{xss_payload}</code>) في الصفحة. قد يستغلها المهاجم."
                ))
                xss_found = True

        except requests.exceptions.RequestException:
            pass # تجاهل الخطأ والمتابعة

        if not xss_found:
            html_results.append(create_report_card(
                "فحص البرمجة عبر المواقع (XSS)",
                "غير موجودة",
                "card-safe",
                "لم يتم العثور على انعكاس مباشر لحمولة XSS النصية."
            ))

        # ======================================================================
        # 4. عرض ملخص النتائج
        # ======================================================================

        display(HTML("".join(html_results)))
        display(HTML(f"<h4><span class='result-good'>الفحص انتهى بنجاح!</span> يرجى مراجعة البطاقات أعلاه.</h4>"))


# ==============================================================================
# 3. ربط الدالة بالزر
# ==============================================================================
scan_button.on_click(run_scan)

Output()

In [ ]:
!pip install requests ipywidgets --quiet

import requests
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from datetime import datetime
import io

# ==============================================================================
# 1. تصميم واجهة المستخدم (GUI)
# ==============================================================================

# تعريف أنماط CSS لتحسين مظهر الواجهة وعرض النتائج
STYLE = """
<style>
    /* الخطوط والألوان العامة */
    @import url('https://fonts.googleapis.com/css2?family=Cairo:wght@400;700&display=swap');
    body, .widget-box {
        font-family: 'Cairo', sans-serif;
    }

    /* العنوان الرئيسي - لون أزرق غامق جديد */
    .title {
        font-size: 32px;
        font-weight: 700;
        color: #1a237e; /* Indigo Dark */
        text-align: center;
        margin: 20px 0;
        border-bottom: 3px solid #1a237e;
        padding-bottom: 10px;
    }

    /* العنوان الفرعي - لون أزرق فاتح جديد */
    .subtitle {
        font-size: 18px;
        color: #5c6bc0; /* Indigo Light */
        text-align: center;
        margin-bottom: 20px;
    }

    /* أنماط النتائج - ألوان أكثر تركيزاً */
    /* أخضر قوي للآمن */
    .result-good {color: #2e7d32; font-weight: 700;}
    /* أحمر قوي للخطير */
    .result-bad {color: #c62828; font-weight: 700;}
    /* برتقالي قوي للتحذير */
    .result-warn {color: #ff8f00; font-weight: 700;}
    /* لون داكن للنصوص الأساسية */
    .result-dark {color: #37474f; font-weight: 700;}

    /* بطاقة النتيجة */
    .scan-card {
        padding: 15px;
        margin: 10px 0;
        border-radius: 8px;
        box-shadow: 0 4px 10px rgba(0,0,0,0.15); /* ظل أوضح */
        border-right: 7px solid; /* حد أسمك */
        transition: transform 0.2s;
        background-color: #ffffff; /* خلفية بيضاء لزيادة التباين */
    }
    .scan-card:hover {
        transform: translateY(-2px);
        box-shadow: 0 6px 12px rgba(0,0,0,0.20);
    }

    /* ألوان الحدود للبطاقات */
    .card-safe { border-right-color: #43a047; }
    .card-danger { border-right-color: #e53935; }
    .card-warning { border-right-color: #fb8c00; }
    .card-info { border-right-color: #1e88e5; }

    .scan-card h4 {
        margin-top: 0;
        border-bottom: 1px dashed #bdbdbd;
        padding-bottom: 5px;
        font-weight: 700;
        color: #37474f; /* أزرق رمادي داكن */
    }

    /* تحسين زر الفحص */
    .widget-button {
        background-color: #3f51b5 !important; /* أزرق نيلي جذاب */
        color: white !important;
        font-weight: bold !important;
        border-radius: 25px !important;
        border: none !important;
        box-shadow: 0 4px 8px rgba(63, 81, 181, 0.4) !important;
    }
</style>
"""

# عرض العناوين باستخدام HTML المخصص
display(HTML(STYLE))
display(HTML('<div class="title">ماسح الثغرات الأمنية للمواقع</div>'))
display(HTML('<div class="subtitle">فحص أولي للثغرات الشائعة: SQLi • XSS • HTTPS</div>'))


# حقل إدخال الرابط
url_input = widgets.Text(
    value='http://testphp.vulnweb.com',
    placeholder='أدخل الرابط الكامل للموقع هنا (مثال: https://example.com)',
    description='رابط الموقع:',
    layout=widgets.Layout(width='90%'),
    style={'description_width': '120px'}
)

# زر الفحص
scan_button = widgets.Button(
    description='بدء الفحص الأمني',
    button_style='info',
    icon='search',
    layout=widgets.Layout(width='90%', height='50px')
)
# إضافة الكلاس المخصص للزر ليتأثر بالتصميم الجديد
scan_button.add_class('widget-button')


# مكان إظهار النتائج
output = widgets.Output()

# تجميع حقل الإدخال والزر في صندوق رأسي
input_box = widgets.VBox([url_input, scan_button], layout=widgets.Layout(align_items='center', margin='10px'))
display(input_box)
display(output)


# ==============================================================================
# 2. منطق الفحص
# ==============================================================================

def create_report_card(title, result, status_class, description):
    """إنشاء بطاقة HTML لعرض نتيجة فحص معين."""
    return f"""
    <div class="scan-card {status_class}">
        <h4>{title} <span style="float: left;"> الحالة: <span class="{status_class.replace('card-', 'result-')}">{result}</span></span></h4>
        <p>{description}</p>
    </div>
    """

def run_scan(b):
    """تشغيل عملية الفحص عند النقر على الزر."""
    with output:
        clear_output()
        url = url_input.value.strip()

        if not url:
            display(HTML('<div class="result-bad">خطأ: الرجاء إدخال رابط صحيح للموقع!</div>'))
            return

        # تصحيح البروتوكول في حال نسيانه
        if not url.startswith(("http://", "https://")):
            url = "http://" + url

        display(HTML(f"<h3><span class='result-warn'>جاري فحص:</span> {url}</h3>"))

        # قائمة لتجميع بطاقات النتائج لعرضها مرة واحدة
        html_results = []

        # ======================================================================
        # 1. فحص الاتصال الأولي و HTTPS
        # ======================================================================

        try:
            # محاولة الوصول عبر HTTPS أولاً
            r = requests.get(url, timeout=15, verify=True, allow_redirects=True)

            # التحقق من أن الموقع يعمل بـ HTTPS بعد إعادة التوجيه
            if r.url.startswith("https://"):
                html_results.append(create_report_card(
                    "فحص أمان الاتصال (HTTPS)",
                    "آمن",
                    "card-safe",
                    "الموقع يعمل ببروتوكول HTTPS المشفر، مما يضمن سرية البيانات المنقولة."
                ))
            else:
                html_results.append(create_report_card(
                    "فحص أمان الاتصال (HTTPS)",
                    "تحذير",
                    "card-warning",
                    "الموقع يستخدم بروتوكول HTTP غير المشفر. قد تكون البيانات عرضة للاعتراض."
                ))

        except requests.exceptions.SSLError:
            # مشكلة في شهادة SSL
            html_results.append(create_report_card(
                "فحص أمان الاتصال (HTTPS)",
                "خطأ في الشهادة",
                "card-danger",
                "تم اكتشاف مشكلة في شهادة SSL (مثل انتهاء الصلاحية أو عدم الموثوقية). يجب إصلاحها فوراً."
            ))
        except requests.exceptions.RequestException as e:
            # أي خطأ آخر في الاتصال (مثل عدم الوصول للموقع)
            html_results.append(create_report_card(
                "فحص الاتصال",
                "فشل",
                "card-danger",
                f"فشل الاتصال بالموقع الهدف. (الخطأ: {str(e)})"
            ))
            # إنهاء الفحص إذا لم يتمكن من الوصول للموقع
            display(HTML("".join(html_results)))
            display(HTML(f"<h4><span class='result-bad'>الفحص توقف:</span> تعذر الوصول للهدف.</h4>"))
            return

        # ======================================================================
        # 2. فحص حقن SQL (SQLi)
        # ======================================================================

        sqli_payloads = ["' OR '1'='1", "1' OR '1'='1' --", "admin' --"]
        sqli_found = False

        # محاولة فحص المسارات الشائعة للاستعلامات
        test_url_base = url.split('?')[0] # إزالة أي معلمات URL موجودة
        test_paths = ['?id=', '?q=', '/product?id=', '/item?q=']

        for p in sqli_payloads:
            for path in test_paths:
                test_url = test_url_base + path + p
                try:
                    resp = requests.get(test_url, timeout=8)
                    # البحث عن كلمات مفتاحية تشير إلى أخطاء في قاعدة البيانات
                    if any(k in resp.text.lower() for k in ["sql", "syntax", "mysql", "error", "warning"]):
                        html_results.append(create_report_card(
                            "فحص حقن SQL (SQLi)",
                            "مكتشفة (خطير)",
                            "card-danger",
                            f"تم اكتشاف ثغرة حقن SQL واضحة! (Payload: <code>{p}</code>). قد يتمكن المهاجم من الوصول لقاعدة البيانات."
                        ))
                        sqli_found = True
                        break # الخروج من حلقة الـ path بعد الإيجاد
                except requests.exceptions.RequestException:
                    continue # تجاهل أخطاء الاتصال للمسارات الفرعية
            if sqli_found:
                break # الخروج من حلقة الـ payload بعد الإيجاد

        if not sqli_found:
            html_results.append(create_report_card(
                "فحص حقن SQL (SQLi)",
                "غير موجودة",
                "card-safe",
                "لم يتم العثور على دلائل واضحة لثغرة SQLi بالحمولة الأولية المستخدمة."
            ))

        # ======================================================================
        # 3. فحص البرمجة عبر المواقع (XSS)
        # ======================================================================

        # حمولة XSS أساسية لاختبار انعكاس النص
        xss_payload = "<script>console.log('XSS_TEST_REFLECTION')</script>"
        xss_test_string = "XSS_TEST_REFLECTION"
        xss_found = False

        # محاولة إدخال الحمولة في مسار افتراضي
        test_url = test_url_base + f"?search={xss_payload}"
        try:
            resp = requests.get(test_url, timeout=8)
            # التحقق مما إذا تم عكس النص داخل محتوى الصفحة
            if xss_test_string in resp.text:
                html_results.append(create_report_card(
                    "فحص البرمجة عبر المواقع (XSS)",
                    "مكتشفة (تحذير)",
                    "card-warning",
                    f"تم اكتشاف انعكاس محتمل لحمولة XSS (<code>{xss_payload}</code>) في الصفحة. قد يستغلها المهاجم."
                ))
                xss_found = True

        except requests.exceptions.RequestException:
            pass # تجاهل الخطأ والمتابعة

        if not xss_found:
            html_results.append(create_report_card(
                "فحص البرمجة عبر المواقع (XSS)",
                "غير موجودة",
                "card-safe",
                "لم يتم العثور على انعكاس مباشر لحمولة XSS النصية."
            ))

        # ======================================================================
        # 4. عرض ملخص النتائج
        # ======================================================================

        display(HTML("".join(html_results)))
        display(HTML(f"<h4><span class='result-good'>الفحص انتهى بنجاح!</span> يرجى مراجعة البطاقات أعلاه.</h4>"))


# ==============================================================================
# 3. ربط الدالة بالزر
# ==============================================================================
scan_button.on_click(run_scan)

Output()